In [ ]:
#the classic, convert from native -> mni, binarize, split hemispheres

In [2]:
import ants
import nilearn
from nilearn import plotting
import os
import shutil
import nibabel as nib
import glob
import numpy as np
from nilearn.image import binarize_img
from nilearn.image import load_img
from nilearn.image import math_img
from nilearn.image import threshold_img
from nilearn.image import resample_to_img

In [ ]:
#convert all lesions from native to MNI
from nipype.interfaces.ants import ApplyTransforms
directory = '/Users/jp1590/Desktop/Cing_Segmentations/segmentations/'
directory = sorted(glob.glob(os.path.join(directory,'Lesion*')))

for filename in directory:
    root = filename.split('/')[-1]
    r_root = root.split('.')[0]
    s_root = root.split('_')[1].split('.')[0]
    at = ApplyTransforms()
    at.inputs.dimension = 3
    at.inputs.input_image = filename
    at.inputs.reference_image = '/Users/jp1590/Desktop/MNI152_T1_0.5mm.nii.gz'
    at.inputs.output_image = f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{s_root}/derivatives/leaddbs/sub-{s_root}/stimulations/{r_root}_output_image.nii'
    at.inputs.transforms = f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{s_root}/derivatives/leaddbs/sub-{s_root}/normalization/transformations/sub-{s_root}_from-anchorNative_to-MNI152NLin2009bAsym_desc-ants.nii.gz'
    at.inputs.invert_transform_flags = [False]
    at.run()

In [166]:
#binarize and threshold all lesions
file = ['3140545','3627630','3715814','4053179','4598723','3523469','3788330','4302726','4621036','3569148','3697607','3769203','3901639','4544470']

for i in file:
    lesion_img = load_img(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{i}/derivatives/leaddbs/sub-{i}/stimulations/Lesion_{i}_output_image.nii')
    mask_edema = math_img('img > 1', img=lesion_img) #for just edema
    mask_edema.to_filename(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{i}/derivatives/leaddbs/sub-{i}/stimulations/sub-{i}_edema_sim-binary_model-simbio.nii')
    threshold = math_img('np.where(img1==1,1,0)',img1=lesion_img) #for just lesion
    nib.save(threshold,os.path.join(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{i}/derivatives/leaddbs/sub-{i}/stimulations/sub-{i}_lesion_sim-binary_model-simbio.nii'))
    mask_both = math_img('img > 0', img=lesion_img) #for both edema and lesion
    mask_both.to_filename(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{i}/derivatives/leaddbs/sub-{i}/stimulations/sub-{i}_lesion_and_edema_sim-binary_model-simbio.nii')

In [72]:
#separate by hemisphere for lesion and edema
l_e_list = [3140545, 4598723, 3523469, 3769203,3901639,4621036,3569148,3697607,4053179,3627630,3715814,3788330,4302726,4544470]

#keep only left hemisphere
for item in l_e_list:
    img_nifti_LH = nib.load(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_and_edema_sim-binary_model-simbio.nii')
    nifti_array_LH = img_nifti_LH.get_fdata()
    nifti_array_LH[180:363,:,:]=0
    new_image_LH = nib.Nifti1Image(nifti_array_LH, affine=img_nifti_LH.affine)
    nib.save(new_image_LH,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_and_edema_L_hemisphere.nii')
    nib.save(new_image_LH,f'/Volumes/Cingulate/Cingulotomy/segmentations/LH/sub-{item}_lesion_and_edema_L_hemisphere.nii')
    
#keep only right hemisphere
for item in l_e_list:
    img_nifti_RH = nib.load(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_and_edema_sim-binary_model-simbio.nii')
    nifti_array_RH = img_nifti_RH.get_fdata()
    nifti_array_RH[0:180,:,:]=0
    new_image_RH = nib.Nifti1Image(nifti_array_RH, affine=img_nifti_RH.affine)
    nib.save(new_image_RH,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_and_edema_R_hemisphere.nii')
    nib.save(new_image_RH,f'/Volumes/Cingulate/Cingulotomy/segmentations/RH/sub-{item}_lesion_and_edema_R_hemisphere.nii')
  

#separate by hemisphere for just edema
e_list = [3140545,3764548,4598723,3523469,3769203,3901639,4621036,3569148,3697607,4053179,3627630,3715814,3788330,4302726,4544470]

#keep only left hemisphere
for item in e_list:
    img_nifti_LH = nib.load(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_edema_sim-binary_model-simbio.nii')
    nifti_array_LH = img_nifti_LH.get_fdata()
    nifti_array_LH[180:363,:,:]=0
    new_image_LH = nib.Nifti1Image(nifti_array_LH, affine=img_nifti_LH.affine)
    nib.save(new_image_LH,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_edema_L_hemisphere.nii')
    nib.save(new_image_LH,f'/Volumes/Cingulate/Cingulotomy/segmentations/LH/sub-{item}_edema_L_hemisphere.nii')
    
#keep only right hemisphere
for item in e_list:
    img_nifti_RH = nib.load(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_edema_sim-binary_model-simbio.nii')
    nifti_array_RH = img_nifti_RH.get_fdata()
    nifti_array_RH[0:180,:,:]=0
    new_image_RH = nib.Nifti1Image(nifti_array_RH, affine=img_nifti_RH.affine)
    nib.save(new_image_RH,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_edema_R_hemisphere.nii')
    nib.save(new_image_RH,f'/Volumes/Cingulate/Cingulotomy/segmentations/RH/sub-{item}_edema_R_hemisphere.nii')
    
    
#separate by hemisphere for just lesion
l_list = [3140545,4423695, 4598723,3523469,3769203,3901639,4621036,3569148,3697607,4053179,3627630,3715814,3788330,4302726,4544470]

#keep only left hemisphere
for item in l_list:
    img_nifti_LH = nib.load(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_sim-binary_model-simbio.nii')
    nifti_array_LH = img_nifti_LH.get_fdata()
    nifti_array_LH[180:363,:,:]=0
    new_image_LH = nib.Nifti1Image(nifti_array_LH, affine=img_nifti_LH.affine)
    nib.save(new_image_LH,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_L_hemisphere.nii')
    nib.save(new_image_LH,f'/Volumes/Cingulate/Cingulotomy/segmentations/LH/sub-{item}_lesion_L_hemisphere.nii')
    
#keep only right hemisphere
for item in l_list:
    img_nifti_RH = nib.load(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_sim-binary_model-simbio.nii')
    nifti_array_RH = img_nifti_RH.get_fdata()
    nifti_array_RH[0:180,:,:]=0
    new_image_RH = nib.Nifti1Image(nifti_array_RH, affine=img_nifti_RH.affine)
    nib.save(new_image_RH,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_R_hemisphere.nii')
    nib.save(new_image_RH,f'/Volumes/Cingulate/Cingulotomy/segmentations/RH/sub-{item}_lesion_R_hemisphere.nii')

In [75]:
#separate cingulotomy and SCT for lesion and edema
cing_sct_list = [4053179, 3569148]

#lesion and edema
for item in cing_sct_list: #cing
    img_nifti_cing_lh = nib.load(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_and_edema_L_hemisphere.nii')
    nifti_array_cing_lh = img_nifti_cing_lh.get_fdata()
    nifti_array_cing_lh[:,:,0:150]=0
    new_image_cing_lh = nib.Nifti1Image(nifti_array_cing_lh, affine=img_nifti_cing_lh.affine)
    nib.save(new_image_cing_lh,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_cing_lesion_and_edema_L_hemisphere.nii')
    
    img_nifti_cing_rh = nib.load(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_and_edema_R_hemisphere.nii')
    nifti_array_cing_rh = img_nifti_cing_rh.get_fdata()
    nifti_array_cing_rh[:,:,0:150]=0
    new_image_cing_rh = nib.Nifti1Image(nifti_array_cing_rh, affine=img_nifti_cing_rh.affine)
    nib.save(new_image_cing_rh,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_cing_lesion_and_edema_R_hemisphere.nii')
    
for item in cing_sct_list: #sct
    img_nifti_sct_rh = nib.load(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_and_edema_R_hemisphere.nii')
    nifti_array_sct_rh = img_nifti_sct_rh.get_fdata()
    nifti_array_sct_rh[:,:,150:363]=0
    new_image_sct_rh = nib.Nifti1Image(nifti_array_sct_rh, affine=img_nifti_sct_rh.affine)
    nib.save(new_image_sct_rh,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_SCT_lesion_and_edema_R_hemisphere.nii')
    
    img_nifti_sct_lh = nib.load(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_and_edema_L_hemisphere.nii')
    nifti_array_sct_lh = img_nifti_sct_lh.get_fdata()
    nifti_array_sct_lh[:,:,150:363]=0
    new_image_sct_lh = nib.Nifti1Image(nifti_array_sct_lh, affine=img_nifti_sct_lh.affine)
    nib.save(new_image_sct_lh,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_SCT_lesion_and_edema_L_hemisphere.nii')

In [74]:
#separate cingulotomy and SCT for lesion
cing_sct_lesion_list = [4053179, 3569148, 4423695, 4459639]

for item in cing_sct_lesion_list: #cing
    img_nifti_cing_lh = nib.load(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_L_hemisphere.nii')
    nifti_array_cing_lh = img_nifti_cing_lh.get_fdata()
    nifti_array_cing_lh[:,:,0:150]=0
    new_image_cing_lh = nib.Nifti1Image(nifti_array_cing_lh, affine=img_nifti_cing_lh.affine)
    nib.save(new_image_cing_lh,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_cing_lesion_L_hemisphere.nii')
    
    img_nifti_cing_rh = nib.load(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_R_hemisphere.nii')
    nifti_array_cing_rh = img_nifti_cing_rh.get_fdata()
    nifti_array_cing_rh[:,:,0:150]=0
    new_image_cing_rh = nib.Nifti1Image(nifti_array_cing_rh, affine=img_nifti_cing_rh.affine)
    nib.save(new_image_cing_rh,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_cing_lesion_R_hemisphere.nii')
    
for item in cing_sct_lesion_list: #sct
    img_nifti_sct_rh = nib.load(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_R_hemisphere.nii')
    nifti_array_sct_rh = img_nifti_sct_rh.get_fdata()
    nifti_array_sct_rh[:,:,150:363]=0
    new_image_sct_rh = nib.Nifti1Image(nifti_array_sct_rh, affine=img_nifti_sct_rh.affine)
    nib.save(new_image_sct_rh,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_SCT_lesion_R_hemisphere.nii')
    
    img_nifti_sct_lh = nib.load(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_lesion_L_hemisphere.nii')
    nifti_array_sct_lh = img_nifti_sct_lh.get_fdata()
    nifti_array_sct_lh[:,:,150:363]=0
    new_image_sct_lh = nib.Nifti1Image(nifti_array_sct_lh, affine=img_nifti_sct_lh.affine)
    nib.save(new_image_sct_lh,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/sub-{item}/derivatives/leaddbs/sub-{item}/stimulations/sub-{item}_SCT_lesion_L_hemisphere.nii')

In [4]:
#subtracting lesion from lesion and edema file to create edema file
from nilearn.image import math_img
subs = os.listdir('/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/')
for i in subs:
    if i.startswith('sub'):
        if os.path.exists(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/{i}/derivatives/leaddbs/{i}/stimulations/{i}_cing_lesion_and_edema_sim-binary_model-simbio.nii')==True:
            cing_lesion_edema = f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/{i}/derivatives/leaddbs/{i}/stimulations/{i}_cing_lesion_and_edema_sim-binary_model-simbio.nii'
            cing_lesion = f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/{i}/derivatives/leaddbs/{i}/stimulations/{i}_cing_lesion_sim-binary_model-simbio.nii'
            cing_result_img = math_img('img1-img2',img1=cing_lesion_edema,img2=cing_lesion)
            nib.save(cing_result_img,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/{i}/derivatives/leaddbs/{i}/stimulations/{i}_cing_edema_sim-binary_model-simbio.nii')
            
for i in subs:
    if i.startswith('sub'):
        if os.path.exists(f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/{i}/derivatives/leaddbs/{i}/stimulations/{i}_SCT_lesion_and_edema_sim-binary_model-simbio.nii')==True:
            sct_lesion_edema = f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/{i}/derivatives/leaddbs/{i}/stimulations/{i}_SCT_lesion_and_edema_sim-binary_model-simbio.nii'
            sct_lesion = f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/{i}/derivatives/leaddbs/{i}/stimulations/{i}_SCT_lesion_sim-binary_model-simbio.nii'
            sct_result_img = math_img('img1-img2',img1=sct_lesion_edema,img2=sct_lesion)
            nib.save(sct_result_img,f'/Users/jp1590/Desktop/Cing_Segmentations/processed_subjects/{i}/derivatives/leaddbs/{i}/stimulations/{i}_SCT_edema_sim-binary_model-simbio.nii')
            
for i in subs:
    if i.startswith('sub'):
        if os.path.exists(f'/Volumes/Cingulate/Cingulotomy/segmentations_processing/{i}/derivatives/leaddbs/{i}/stimulations/{i}_cing_lesion_and_edema_sim-binary_model-simbio.nii')==True:
            cing_lesion_edema = f'/Volumes/Cingulate/Cingulotomy/segmentations_processing/{i}/derivatives/leaddbs/{i}/stimulations/{i}_cing_lesion_and_edema_sim-binary_model-simbio.nii'
            cing_lesion = f'/Volumes/Cingulate/Cingulotomy/segmentations_processing/{i}/derivatives/leaddbs/{i}/stimulations/{i}_cing_lesion_sim-binary_model-simbio.nii'
            cing_result_img = math_img('img1-img2',img1=cing_lesion_edema,img2=cing_lesion)
            nib.save(cing_result_img,f'/Volumes/Cingulate/Cingulotomy/segmentations_processing/{i}/derivatives/leaddbs/{i}/stimulations/{i}_cing_edema_sim-binary_model-simbio.nii')
            
for i in subs:
    if i.startswith('sub'):
        if os.path.exists(f'/Volumes/Cingulate/Cingulotomy/segmentations_processing/{i}/derivatives/leaddbs/{i}/stimulations/{i}_SCT_lesion_and_edema_sim-binary_model-simbio.nii')==True:
            sct_lesion_edema = f'/Volumes/Cingulate/Cingulotomy/segmentations_processing/{i}/derivatives/leaddbs/{i}/stimulations/{i}_SCT_lesion_and_edema_sim-binary_model-simbio.nii'
            sct_lesion = f'/Volumes/Cingulate/Cingulotomy/segmentations_processing/{i}/derivatives/leaddbs/{i}/stimulations/{i}_SCT_lesion_sim-binary_model-simbio.nii'
            sct_result_img = math_img('img1-img2',img1=sct_lesion_edema,img2=sct_lesion)
            nib.save(sct_result_img,f'/Volumes/Cingulate/Cingulotomy/segmentations_processing/{i}/derivatives/leaddbs/{i}/stimulations/{i}_SCT_edema_sim-binary_model-simbio.nii')

In [19]:
#make efields from binary mask
files = os.listdir('/Volumes/Cingulate/Cingulotomy/segmentations/sct/z1-4')
for i in files:
    if os.path.exists(f'/Volumes/Cingulate/Cingulotomy/segmentations/resample/{i[0:11]}_ses-postop_space-MNI152NLin2009bAsym_desc-preproc_acq-ax_MRI.nii'):
        binary = load_img(f'/Volumes/Cingulate/Cingulotomy/segmentations/sct/z1-4/{i}')
        original = load_img(f'/Volumes/Cingulate/Cingulotomy/segmentations/resample/{i[0:11]}_ses-postop_space-MNI152NLin2009bAsym_desc-preproc_acq-ax_MRI.nii') 
        efield = math_img('img1*img2',img1=original,img2=binary)
        nib.save(efield,f'/Volumes/Cingulate/Cingulotomy/processed_subjects/{i[0:11]}/derivatives/leaddbs/{i[0:11]}/stimulations/efields/{i[0:11]}-ax_MRI_sct_z1-3.nii')
    
    if os.path.exists(f'/Volumes/Cingulate/Cingulotomy/segmentations/resample/{i[0:11]}_ses-postop_space-MNI152NLin2009bAsym_desc-preproc_acq-sag_MRI.nii'):
        binary = load_img(f'/Volumes/Cingulate/Cingulotomy/segmentations/sct/z1-4/{i}')
        original = load_img(f'/Volumes/Cingulate/Cingulotomy/segmentations/resample/{i[0:11]}_ses-postop_space-MNI152NLin2009bAsym_desc-preproc_acq-sag_MRI.nii') 
        efield = math_img('img1*img2',img1=original,img2=binary)
        nib.save(efield,f'/Volumes/Cingulate/Cingulotomy/processed_subjects/{i[0:11]}/derivatives/leaddbs/{i[0:11]}/stimulations/efields/{i[0:11]}-sag_MRI_sct_z1-3.nii')
        
    if os.path.exists(f'/Volumes/Cingulate/Cingulotomy/segmentations/resample/{i[0:11]}_ses-postop_space-MNI152NLin2009bAsym_desc-preproc_acq-cor_MRI.nii'):
        binary = load_img(f'/Volumes/Cingulate/Cingulotomy/segmentations/sct/z1-4/{i}')
        original = load_img(f'/Volumes/Cingulate/Cingulotomy/segmentations/resample/{i[0:11]}_ses-postop_space-MNI152NLin2009bAsym_desc-preproc_acq-cor_MRI.nii') 
        efield = math_img('img1*img2',img1=original,img2=binary)
        nib.save(efield,f'/Volumes/Cingulate/Cingulotomy/processed_subjects/{i[0:11]}/derivatives/leaddbs/{i[0:11]}/stimulations/efields/{i[0:11]}-cor_MRI_sct_z1-3.nii')